# **Train instance segmentation (YOLOv11)**

In [1]:
import torch

# CUDAが使えるかどうかを確認
cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")


CUDA is available! GPU: NVIDIA GeForce RTX 3080 Ti Laptop GPU


# **Extract_annotations**

cvat形式（xml）で抜き出したアノーテーションをyolo形式に直す

In [19]:
import os
import shutil


xml_path = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\annotations_1-139.xml"
label_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139"
start_id = 1
end_id = 500

# labels_dirが存在する場合、それを削除する
if os.path.exists(label_dir):
    print(f"ディレクトリ '{label_dir}' が存在します。削除します。")
    shutil.rmtree(label_dir)
    print(f"ディレクトリ '{label_dir}' を削除しました。")

# labels_dirを作成する
print(f"ディレクトリ '{label_dir}' を作成します。")
os.makedirs(label_dir)
print(f"ディレクトリ '{label_dir}' を作成しました。")




ディレクトリ 'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139' が存在します。削除します。
ディレクトリ 'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139' を削除しました。
ディレクトリ 'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139' を作成します。
ディレクトリ 'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139' を作成しました。


In [20]:

import xml.etree.ElementTree as ET
import os
import shutil

# CVAT形式のXMLアノテーションファイルを解析し、指定されたID範囲の画像について、
# ポリゴンアノテーションをYOLO形式（クラスID + 正規化された座標）に変換し、
# 対応する画像ファイル名を持つテキストファイルとして保存するスクリプト。

# --- Configuration ---
xml_path = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\annotations_1-139.xml"
label_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139"
start_id = 1   # Inclusive start image ID
end_id = 500   # Inclusive end image ID
# -------------------

# --- Directory Handling ---
# labels_dirが存在する場合、それを削除する
if os.path.exists(label_dir):
    print(f"ディレクトリ '{label_dir}' が存在します。削除します。")
    shutil.rmtree(label_dir)
    print(f"ディレクトリ '{label_dir}' を削除しました。")

# labels_dirを作成する
print(f"ディレクトリ '{label_dir}' を作成します。")
os.makedirs(label_dir)
print(f"ディレクトリ '{label_dir}' を作成しました。")



# --- XML Parsing and Conversion ---
try:
    print(f"Parsing XML file: {xml_path}")
    tree = ET.parse(xml_path)
    root = tree.getroot()
except FileNotFoundError:
    print(f"Error: XML file not found at {xml_path}")
    exit(1)
except ET.ParseError as e:
    print(f"Error parsing XML file: {e}")
    exit(1)

# 1. Create Label to ID mapping from the <labels> section
label_to_id = {}
label_elements = root.findall('.//meta/task/labels/label')
if not label_elements:
     label_elements = root.findall('.//labels/label') # Try alternative path if needed

if not label_elements:
    print("Warning: Could not find <label> elements in the XML meta section. Will build mapping dynamically.")
else:
    for idx, label_tag in enumerate(label_elements):
        name = label_tag.find('name').text
        if name not in label_to_id:
            label_to_id[name] = idx
            print(f"Mapped label '{name}' to class ID {idx}")

id_counter = len(label_to_id) # Start dynamic mapping from the next available ID


print(f"\nProcessing images with ID from {start_id} to {end_id}...")
images_processed = 0
images_skipped_out_of_range = 0

# 2. Iterate through images and process annotations
for image in root.findall('.//image'):
    image_id_str = image.get('id')
    image_name = image.get('name')
    width_str = image.get('width')
    height_str = image.get('height')

    if not all([image_id_str, image_name, width_str, height_str]):
        print(f"Warning: Skipping image entry due to missing attributes (id, name, width, or height).")
        continue

    try:
        image_id = int(image_id_str)
        img_width = int(width_str)
        img_height = int(height_str)
    except ValueError:
        print(f"Warning: Skipping image '{image_name}' due to invalid numeric attribute (id, width, or height).")
        continue

    # Check if image ID is within the desired range
    if not (start_id <= image_id <= end_id):
        images_skipped_out_of_range += 1
        continue

    if img_width <= 0 or img_height <= 0:
        print(f"Warning: Skipping image '{image_name}' (ID: {image_id}) due to zero or negative width/height ({img_width}x{img_height}).")
        continue

    # Prepare output file path
    base_name = os.path.splitext(os.path.basename(image_name))[0]
    output_txt_path = os.path.join(label_dir, f"{base_name}.txt")
    yolo_annotations = []

    # Process polygons for the current image
    for polygon in image.findall('.//polygon'):
        label = polygon.get('label')
        points_str = polygon.get('points')

        if not label or not points_str:
            print(f"Warning: Skipping polygon in image '{image_name}' (ID: {image_id}) due to missing label or points.")
            continue

        # Assign class ID, dynamically add if not seen before
        if label not in label_to_id:
             if not label_elements: # Only add dynamically if initial mapping failed
                 label_to_id[label] = id_counter
                 print(f"Dynamically mapped new label '{label}' to class ID {id_counter}")
                 id_counter += 1
             else:
                 print(f"Warning: Label '{label}' found in image '{image_name}' (ID: {image_id}) but not defined in <labels> section. Skipping this polygon.")
                 continue # Skip if label mapping should be strict based on <labels>

        class_id = label_to_id[label]

        # Parse and normalize points
        normalized_points = []
        try:
            point_pairs = points_str.split(';')
            if len(point_pairs) < 3: # A polygon needs at least 3 points
                 print(f"Warning: Skipping polygon with label '{label}' in image '{image_name}' (ID: {image_id}) - less than 3 points ({len(point_pairs)} points found).")
                 continue

            for pair in point_pairs:
                x_str, y_str = pair.split(',')
                x = float(x_str)
                y = float(y_str)

                # Clamp coordinates to be within image bounds before normalizing
                x = max(0.0, min(x, img_width))
                y = max(0.0, min(y, img_height))

                norm_x = x / img_width
                norm_y = y / img_height
                normalized_points.extend([f"{norm_x:.6f}", f"{norm_y:.6f}"]) # Format with 6 decimal places

        except ValueError:
            print(f"Warning: Skipping polygon with label '{label}' in image '{image_name}' (ID: {image_id}) due to invalid point format in '{points_str}'.")
            continue
        except Exception as e:
             print(f"Warning: Skipping polygon with label '{label}' in image '{image_name}' (ID: {image_id}) due to error processing points '{points_str}': {e}")
             continue

        # Format the YOLO line
        yolo_line = f"{class_id} {' '.join(normalized_points)}"
        yolo_annotations.append(yolo_line)

    # Write annotations to the output file
    if yolo_annotations:
        try:
            with open(output_txt_path, 'w') as f:
                for line in yolo_annotations:
                    f.write(line + '\n')
            images_processed += 1
            if images_processed % 50 == 0: # Print progress every 50 images
                 print(f"  Processed {images_processed} images...")
        except IOError as e:
            print(f"Error writing to file {output_txt_path}: {e}")
    # else:
        # Optionally print a message if an image in range had no valid annotations
        # print(f"Image '{image_name}' (ID: {image_id}) had no valid polygons to write.")


# --- Final Summary ---
print("\n--- Conversion Summary ---")
print(f"Total images processed and files created: {images_processed}")
print(f"Total images skipped (outside ID range {start_id}-{end_id}): {images_skipped_out_of_range}")
print(f"Label mapping used: {label_to_id}")
print(f"Output files saved in: {label_dir}")
print("--------------------------")

ディレクトリ 'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139' が存在します。削除します。
ディレクトリ 'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139' を削除しました。
ディレクトリ 'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139' を作成します。
ディレクトリ 'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139' を作成しました。
Parsing XML file: C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\annotations_1-139.xml
Mapped label 'Eyelid' to class ID 0
Mapped label 'Caruncle' to class ID 1

Processing images with ID from 1 to 500...
  Processed 50 images...
  Processed 100 images...
  Processed 150 images...
  Processed 200 images...
  Processed 250 images...
  Processed 300 images...
  Processed 350 images...
  Processed 400 images...
  Processed 450 images...

--- Conversion Summary ---
Total images processed and files created: 497
Total images skipped (outside ID range 1-500): 3967
Label mapping used: {'Eyelid': 0, 'Caruncle':

# **Training**

```

最終形
project_root/
│
└── data/
    |
    │
    ├── train/
    │   ├── images/       # トレーニング用の画像
    │   │   ├── img1.jpg
    │   │   ├── img2.jpg
    │   │   └── ...
    │   │
    │   └── labels/      # トレーニング用のラベル（アノテーション）
    │       ├── img1.txt
    │       ├── img2.txt
    │       └── ...
    │
    ├── val/
    │   ├── images/      # 検証用の画像
    │   │   ├── img1.jpg
    │   │   └── ...
    │   │
    │   └── labels/     # 検証用のラベル
    │       ├── img1.txt
    │       └── ...
    │
    ├── test/           # (オプション) テスト用のデータセット
    |   ├── images/
    |   └── labels/
    │
    ├── Train.txt           # トレーニングファイルのリスト
    │
    └── dataset.yaml        # データセット設定ファイル:

    ```

dataset.yaml
# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ./data/images/train
val: ./data/images/valid

# number of classes
nc: 2

# class names:
  0: Eyelid
  1: Caruncle

In [29]:
# このスクリプトは、指定されたディレクトリ内のラベルファイルと画像ファイルを
# 以下のようにコピーします：
#
# 1. source_labels_dir/LABELS_xxxx/ 内の全ファイル → destination_labels_dir/
# 2. source_images_dir/IMG_xxxx/ 内の全ファイル → destination_images_dir/
#
# 処理対象のディレクトリは、label_dirとimage_dirのリストで指定します。
# 各ディレクトリ内のファイルは全て、指定された出力先ディレクトリにコピーされます。
# tqdmを使用して、コピーの進捗状況をプログレスバーで表示します。

import shutil
import os
from tqdm import tqdm

# コピー元ディレクトリ
source_labels_dir = r'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\labels_1-139'
source_images_dir = r'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\images'

# コピー先ディレクトリ
destination_labels_dir = r'C:\Users\CorneAI\YOLOv11_Mobius\data\labels'
destination_images_dir = r'C:\Users\CorneAI\YOLOv11_Mobius\data\images'

# 処理対象のディレクトリリスト
# label_dir = ['LABELS_1703_lens_seg']
# image_dir = ['IMG_1703_lens']


# destination_labels_dirとdestination_images_dirが存在する場合は削除し、新規作成する
if os.path.exists(destination_labels_dir):
    shutil.rmtree(destination_labels_dir)
os.makedirs(destination_labels_dir, exist_ok=True)

if os.path.exists(destination_images_dir):
    shutil.rmtree(destination_images_dir)
os.makedirs(destination_images_dir, exist_ok=True)

# ラベルファイルのコピー
# source_labels_dir 内の全ファイルを取得しコピー
if os.path.isdir(source_labels_dir):
    # source_labels_dir 直下のファイルのみをリストアップ
    files_to_copy = [f for f in os.listdir(source_labels_dir) if os.path.isfile(os.path.join(source_labels_dir, f))]
    # tqdm を使って進捗を表示しながらファイルをコピー
    for file in tqdm(files_to_copy, desc="ラベルファイルのコピー中"):
        src = os.path.join(source_labels_dir, file)
        dst = os.path.join(destination_labels_dir, file)
        try:
            shutil.copy(src, dst)
            # print(f"コピーしました: {src} -> {dst}") # tqdmがあるのでコメントアウト推奨
        except Exception as e:
            # エラーハンドリングを追加
            print(f"エラー: {src} のコピー中にエラーが発生しました - {e}")
else:
    # ソースディレクトリが存在しない場合のエラーメッセージ
    print(f"ソースディレクトリが見つかりません: {source_labels_dir}")

# 画像ファイルのコピー
# source_images_dir 内の全ファイルを取得
if os.path.isdir(source_images_dir):
    files_to_copy = [f for f in os.listdir(source_images_dir) if os.path.isfile(os.path.join(source_images_dir, f))]
    for file in tqdm(files_to_copy, desc="画像ファイルのコピー中"):
        src = os.path.join(source_images_dir, file)
        dst = os.path.join(destination_images_dir, file)
        try:
            shutil.copy(src, dst)
            # print(f"コピーしました: {src} -> {dst}") # tqdmがあるのでコメントアウト推奨
        except Exception as e:
            print(f"エラー: {src} のコピー中にエラーが発生しました - {e}")
else:
    print(f"ソースディレクトリが見つかりません: {source_images_dir}")

画像ファイルのコピー中: 100%|██████████| 4467/4467 [00:04<00:00, 1084.33it/s]


In [ ]:
#segmentationはcvatのラベルが1スタートなので、0スタートに修正
"""
# ラベルインデックス調整スクリプト
# 目的：YOLOフォーマットのアノテーションファイルのラベルインデックスを1始まりから0始まりに変更
# 対象：指定されたディレクトリ内のすべての.txtファイル
# 
# 入力ファイルの例：
# 1 0.399550 0.361382 0.482096 0.225870 ...  # ラベル1で始まる
# 2 0.452748 0.391491 0.488882 0.403540 ...  # ラベル2で始まる
#
# 出力ファイルの例：
# 0 0.399550 0.361382 0.482096 0.225870 ...  # ラベル0に変更
# 1 0.452748 0.391491 0.488882 0.403540 ...  # ラベル1に変更
#
# 使用方法：
# 1. ラベルファイルが含まれているディレクトリパスを設定
# 2. スクリプトを実行すると、すべての.txtファイルを処理
# 3. 元のファイルが直接更新されます
# 4. もともと0始まりの場合はskipしてください
#
# 注意：元のファイルが直接更新されるため、実行前にファイルのバックアップを
# 取ることをお勧めします
"""

# import os
# from pathlib import Path

# def adjust_labels(input_path):
#     """
#     ラベルファイルを読み込み、すべてのラベルインデックスを1減らします
#     （1始まりから0始まりに変更）。
#     元のファイルを調整後の内容で上書きします。
    
#     引数：
#         input_path (str): 入力ラベルファイルのパス
#     """
#     # 元のファイル内容を読み込み
#     with open(input_path, 'r') as f:
#         lines = f.readlines()
    
#     adjusted_lines = []
#     for line in lines:
#         parts = line.strip().split()
#         if parts:  # 空行をスキップ
#             # ラベル（最初の数字）を1減らす
#             adjusted_line = f"{int(parts[0]) - 1} {' '.join(parts[1:])}"
#             adjusted_lines.append(adjusted_line)
    
#     # 同じファイルに書き戻す
#     with open(input_path, 'w') as f:
#         f.write('\n'.join(adjusted_lines))

# def process_directory(directory_path):
#     """
#     指定されたディレクトリ内のすべての.txtファイルを処理します。
    
#     引数：
#         directory_path (str): ラベルファイルを含むディレクトリのパス
#     """
#     directory = Path(directory_path)
#     txt_files = list(directory.glob('*.txt'))
#     total_files = len(txt_files)
    
#     print(f"{directory_path} 内で {total_files} 個の.txtファイルが見つかりました")
    
#     for i, file_path in enumerate(txt_files, 1):
#         try:
#             adjust_labels(str(file_path))
#             print(f"処理完了 [{i}/{total_files}]: {file_path.name}")
#         except Exception as e:
#             print(f"エラーが発生しました {file_path.name}: {str(e)}")

# # ディレクトリパス
# labels_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\data\labels"

# # ディレクトリ内のすべてのファイルを処理
# process_directory(labels_dir)
# print("\n処理が完了しました！")

In [30]:
# データの前処理
# 目的: 画像ファイルとラベルファイルの対応関係を確認
# 処理内容:
# 1. ラベルディレクトリとイメージディレクトリのパスを設定
# 2. 各ディレクトリからファイル名(拡張子なし)を取得
# 3. ラベルが存在しない画像ファイルを特定
# 4. 不一致の数を表示

import os

# ラベルとイメージのディレクトリパス
labels_dir = r"data\labels"
images_dir = r"data\images"

# ラベルファイルの basename (拡張子なし) を取得
label_files = {os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt')}

# 画像ファイルの basename (拡張子なし) を取得
image_files = {os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpg')}

# ラベルが存在しない画像ファイルを見つける
images_without_labels = image_files - label_files

# 結果を表示
print(f"ラベルが存在しない画像ファイルの数: {len(images_without_labels)}")

ラベルが存在しない画像ファイルの数: 3970


In [ ]:
# ラベルが存在しない画像ファイルを削除するバージョン(ほぼすべての画像に対象物がある場合)
for img_name in images_without_labels:
    img_path = os.path.join(images_dir, img_name + '.jpg')
    try:
        os.remove(img_path)
        print(f"削除しました: {img_path}")
    except OSError as e:
        print(f"削除に失敗しました {img_path}: {e}")

print("不一致ファイルの削除が完了しました")

In [32]:
# 1. 指定されたディレクトリから画像ファイル(.jpg, .png, .tif)を取得
# 2. 画像ファイルをランダムに並び替え
# 3. train/validディレクトリを作成
# 4. データを8:2の比率で分割
# 5. 画像ファイルと対応するラベルファイル(.txt)を適切なディレクトリに移動

import os
import shutil
import random
from tqdm import tqdm

# データディレクトリのパス設定
data_img_dir = r"data\images"
data_label_dir = r"data\labels"
src_img_dir = r"data\images"
src_label_dir = r"data\labels"

# ソースディレクトリの画像ファイルを直接取得（サブディレクトリを除く）
image_files = [f for f in os.listdir(src_img_dir) 
               if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.tiff')) 
               and os.path.isfile(os.path.join(src_img_dir, f))]

if not image_files:
    print("エラー: 画像ファイルが見つかりません")
else:
    print(f"見つかった画像ファイル数: {len(image_files)}")
    
    # train/validディレクトリの作成
    for split in ['train', 'valid']:
        img_split_dir = os.path.join(data_img_dir, split)
        label_split_dir = os.path.join(data_label_dir, split)
        
        os.makedirs(img_split_dir, exist_ok=True)
        os.makedirs(label_split_dir, exist_ok=True)

    # データの分割
    random.shuffle(image_files)
    train_size = int(len(image_files) * 0.8)
    train_files = image_files[:train_size]
    valid_files = image_files[train_size:]

    # ファイルの移動関数
    def move_files(files, split):
        for img_file in tqdm(files, desc=f"{split}データの移動"):
            try:
                # 画像ファイルの移動
                src_img_path = os.path.join(src_img_dir, img_file)
                dst_img_path = os.path.join(data_img_dir, split, img_file)
                
                if os.path.exists(src_img_path):
                    shutil.move(src_img_path, dst_img_path)

                # 対応するラベルファイルの移動
                label_file = os.path.splitext(img_file)[0] + '.txt'
                src_label_path = os.path.join(src_label_dir, label_file)
                dst_label_path = os.path.join(data_label_dir, split, label_file)
                
                if os.path.exists(src_label_path):
                    shutil.move(src_label_path, dst_label_path)

            except Exception as e:
                print(f"エラーが発生しました ({img_file}): {str(e)}")

    # trainとvalidそれぞれにファイルを移動
    move_files(train_files, 'train')
    move_files(valid_files, 'valid')

    print(f"訓練データ数: {len(train_files)}")
    print(f"検証データ数: {len(valid_files)}")
    print("データの分割と移動が完了しました")

見つかった画像ファイル数: 497


trainデータの移動:   0%|          | 0/397 [00:00<?, ?it/s]

validデータの移動: 100%|██████████| 100/100 [00:00<00:00, 1017.86it/s]

訓練データ数: 397
検証データ数: 100
データの分割と移動が完了しました


In [33]:
import os

# ディレクトリのパス
train_images_dir = r'C:\Users\CorneAI\YOLOv11_Mobius\data\images\train'
train_labels_dir = r'C:\Users\CorneAI\YOLOv11_Mobius\data\labels\train'
val_images_dir = r'C:\Users\CorneAI\YOLOv11_Mobius\data\images\valid'
val_labels_dir = r'C:\Users\CorneAI\YOLOv11_Mobius\data\labels\valid'

# 対応の確認と削除
def check_and_remove_unmatched_files(images_dir, labels_dir):
    image_files = set(os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpg'))
    label_files = set(os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt'))

    unmatched_images = image_files - label_files
    unmatched_labels = label_files - image_files

    # 対応のない画像ファイルを削除
    for image in unmatched_images:
        image_path = os.path.join(images_dir, image + '.jpg')
        if os.path.exists(image_path):
            print(f"画像ファイル {image_path} に対応するラベルがありません。削除しますか？ (y/n): ", end="")
            if input().strip().lower() == 'y':
                os.remove(image_path)
                print(f"{image_path} を削除しました。")

    # 対応のないラベルファイルを削除
    for label in unmatched_labels:
        label_path = os.path.join(labels_dir, label + '.txt')
        if os.path.exists(label_path):
            print(f"ラベルファイル {label_path} に対応する画像がありません。削除しますか？ (y/n): ", end="")
            if input().strip().lower() == 'y':
                os.remove(label_path)
                print(f"{label_path} を削除しました。")

# 訓練データと検証データの対応確認
check_and_remove_unmatched_files(train_images_dir, train_labels_dir)
check_and_remove_unmatched_files(val_images_dir, val_labels_dir)

### **YAMLファイルを作成 (for training)**

In [34]:
import yaml
import os

# data = {
#     'train': r'C:\Users\ykita\ROP_AI_project\data\images\train',
#     'val': r'C:\Users\ykita\ROP_AI_project\data\images\valid',
#     'nc': 1,
#     'names': {
#         0: 'Lens',
#     }
# }

data = {
    'train': r'C:\Users\CorneAI\YOLOv11_Mobius\data\images\train',
    'val': r'C:\Users\CorneAI\YOLOv11_Mobius\data\images\valid',
    'nc': 2,
    'names': {
        0: 'Eyelid',
        1: 'Caruncle',
    }
}

yaml_path = 'data/data.yaml'

# dataディレクトリが存在しない場合は作成
os.makedirs('data', exist_ok=True)

# ファイルの存在確認
if os.path.exists(yaml_path):
    print(f"既存の {yaml_path} を上書きします。")
else:
    print(f"新規に {yaml_path} を作成します。")

# YAMLファイルの書き出し（既存ファイルは上書き）
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.safe_dump(data, f, sort_keys=False, allow_unicode=True)

print("処理が完了しました。")

既存の data/data.yaml を上書きします。
処理が完了しました。


# **Training**

In [35]:
from ultralytics import YOLO

# Load a pretrained YOLO11 segment model
model = YOLO("yolo11x-seg.pt")

# Train the model (eyelid, iris, pupil)
yaml_path = r"C:\Users\CorneAI\YOLOv11_Mobius\data\data.yaml"
results = model.train(data=yaml_path, epochs=1000, imgsz=640, patience=100, batch=0.70, degrees=10)


New https://pypi.org/project/ultralytics/8.3.119 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.53  Python-3.10.5 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3080 Ti Laptop GPU, 16384MiB)
engine\trainer: task=segment, mode=train, model=yolo11x-seg.pt, data=C:\Users\CorneAI\YOLOv11_Mobius\data\data.yaml, epochs=1000, time=None, patience=100, batch=0.7, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, 

train: Scanning C:\Users\CorneAI\YOLOv11_Mobius\data\labels\train... 397 images, 0 backgrounds, 0 corrupt: 100%|██████████| 397/397 [00:00<00:00, 550.77it/s]

train: New cache created: C:\Users\CorneAI\YOLOv11_Mobius\data\labels\train.cache


AutoBatch: Computing optimal batch size for imgsz=640 at 70.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3080 Ti Laptop GPU) 16.00G total, 1.03G reserved, 1.01G allocated, 13.97G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    62052566       319.7         3.985         174.3           nan        (1, 3, 640, 640)                    list
    62052566       639.4         7.095         90.22           nan        (2, 3, 640, 640)                    list
    62052566        1279        12.143         132.7           nan        (4, 3, 640, 640)                    list
    62052566        2557        23.004         227.7           nan        (8, 3, 640, 640)                    list
    62052566        5115        42.035     1.487e+04           nan       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 3 for CUDA:0 11.73G/16.00G (73%) 


train: Scanning C:\Users\CorneAI\YOLOv11_Mobius\data\labels\train.cache... 397 images, 0 backgrounds, 0 corrupt: 100%|██████████| 397/397 [00:00<?, ?it/s]
val: Scanning C:\Users\CorneAI\YOLOv11_Mobius\data\labels\valid... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 366.15it/s]


...

   648/1000      5.24G     0.2996     0.3638     0.1865     0.8654          2        640: 100%|██████████| 133/133 [00:30<00:00,  4.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  6.11it/s]

                   all        100        186      0.958      0.948      0.971      0.823       0.97       0.96      0.976      0.819
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 548, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



648 epochs completed in 6.396 hours.
Optimizer stripped from runs\segment\train5\weights\last.pt, 124.9MB
Optimizer stripped from runs\segment\train5\weights\best.pt, 124.9MB

Validating runs\segment\train5\weights\best.pt...
Ultralytics 8.3.53  Python-3.10.5 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3080 Ti Laptop GPU, 16384MiB)
YOLO11x-seg summary (fused): 491 layers, 62,004,438 parameters, 0 gradients, 318.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  6.45it/s]


                   all        100        186      0.957      0.966       0.98      0.833      0.957      0.966       0.98      0.833
                Eyelid        100        100      0.995       0.99      0.994      0.974      0.995       0.99      0.994      0.986
              Caruncle         86         86      0.919      0.942      0.965      0.692      0.919      0.942      0.965       0.68
Speed: 0.4ms preprocess, 17.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\segment\train5


In [ ]:
from ultralytics import YOLO
# Resume training
# Load a model
model = YOLO(r"C:\Users\CorneAI\YOLOv11_Mobius\instanceenv\runs\segment\train15\weights\last.pt")  # load a partially trained model
results = model.train(resume=True)

# **Inference**

In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the model
model = YOLO(r"C:\Users\CorneAI\YOLOv11_Mobius\models\best_eyelid_184epochs.pt")
# model = RTDETR(r"C:\Users\CorneAI\FacePhoto_instance\models\1-295_538epochs_rtdetr-l.pt")

# Color list (different colors for each class)
colors = [(0, 0, 200), (0, 200, 0), (200, 0, 0), (200, 200, 0), (0, 200, 200), (200, 0, 200)]
linewidth = 3
fontScale = 0.8
fontFace = cv2.FONT_HERSHEY_SIMPLEX
thickness = 2

# Mask overlay function
def overlay(image, mask, color, alpha):
    """Overlay segmentation mask on the image"""
    # Create colored mask with 3 channels
    colored_mask = np.zeros_like(image)
    colored_mask[:] = color
    
    # Apply mask
    image_overlay = np.where(mask[:, :, None] == 1, colored_mask, image)
    
    # Apply transparency
    image_combined = cv2.addWeighted(image, 1 - alpha, image_overlay, alpha, 0)
    return image_combined

# Bounding box and label drawing function
def draw_label(box, img, color, label, line_thickness=3):
    """Draw bounding box and label on the image"""
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    # Draw label background
    text_size = cv2.getTextSize(
        label, fontFace, fontScale=fontScale, thickness=line_thickness
    )[0]
    cv2.rectangle(
        img, (x1, y1), (x1 + text_size[0], y1 - text_size[1] - 2), color, -1
    )  # fill
    
    # Draw label text
    cv2.putText(
        img,
        label,
        (x1, y1 - 3),
        fontFace,
        fontScale,
        [225, 255, 255],
        thickness=line_thickness,
        lineType=cv2.LINE_AA,
    )

    # Draw bounding box
    cv2.rectangle(
        img,
        (x1, y1),
        (x2, y2),
        color,
        linewidth,
    )

# Main processing function
def main():
    # Image path for inference
    image_path = r'C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295\images\2-20010410-53-161556_35d029be5c8e22bfc887f05cde2c5d79469a3d0681b69ac6e021a8e7b5b94f38.JPG'
    
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Get image dimensions
    h, w, _ = image.shape
    
    # Run inference with the model
    results = model(image_path, conf=0.25, iou=0.45)
    result = results[0]
    
    # Create result image (copy of original)
    output_image = image_rgb.copy()
    
    # Process if masks are available
    if result.masks is not None:
        # Confidence scores list
        conf_list = result.boxes.conf.tolist()
        
        # Process each instance
        for i, (mask, box) in enumerate(zip(result.masks.data.cpu().numpy(), result.boxes.data.cpu().numpy())):
            # Resize mask to match image dimensions
            mask = cv2.resize(mask, (w, h))
            
            # Get class ID
            class_id = int(box[5]) if box.shape[0] > 5 else 0
            
            # Select color (cycle through colors if class_id exceeds color list length)
            color_idx = class_id % len(colors)
            
            # Overlay mask with semi-transparency (alpha=0.5)
            output_image = overlay(output_image, mask, colors[color_idx], 0.5)
            
            # Bounding box information
            bbox = box[:4]
            
            # Get class name
            class_name = result.names[class_id] if class_id in result.names else f"Class {class_id}"
            
            # Get confidence score
            score = box[4] if box.shape[0] > 4 else conf_list[i]
            
            # Create label text
            label_text = f"{class_name} {score:.2f}"
            
            # Draw bounding box and label
            draw_label(bbox, output_image, colors[color_idx], label_text, line_thickness=2)
            
            # Output in YOLO format
            x_center = (bbox[0] + bbox[2]) / (2 * w)
            y_center = (bbox[1] + bbox[3]) / (2 * h)
            width = (bbox[2] - bbox[0]) / w
            height = (bbox[3] - bbox[1]) / h
            print(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
        
        # Display results
        plt.figure(figsize=(14, 10))
        plt.subplot(1, 2, 1)
        plt.title("Original Image")
        plt.imshow(image_rgb)
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.title("Instance Segmentation Result")
        plt.imshow(output_image)
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Save results (optional)
        # output_filename = "segmentation_result.jpg"
        # cv2.imwrite(output_filename, cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))
        # print(f"Saved: {output_filename}")
    else:
        print("No segmentation masks found.")
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

# Execute
if __name__ == "__main__":
    main()